In [1]:
import os
import warnings
warnings.simplefilter('ignore')

import geopandas as gpd
import pandas as pd
import numpy as np

In [30]:
def func_hazard_tract_agg(hazard, boundary, hazard_var, boundary_code):

    boundary = boundary.to_crs(epsg = 22523)

    hazard_id = hazard[[hazard_var, 'geometry']]

    boundary_id = boundary[[boundary_code, 'geometry']]

    hazard_census = gpd.overlay(boundary_id, hazard_id, how = 'intersection', keep_geom_type = False)

    hazard_census_max = hazard_census.dissolve(by = boundary_code, aggfunc = 'max').reset_index()

    hazard_census_max = hazard_census_max.rename({hazard_var:hazard_var + '_max'}, axis = 1)

    hazard_census['area'] = hazard_census.geometry.area

    hazard_census[hazard_var + '_mean'] = hazard_census[hazard_var]*hazard_census['area']

    hazard_census_sum = hazard_census.dissolve(by = boundary_code, aggfunc = 'sum').reset_index()

    hazard_census_sum[hazard_var + '_mean'] = hazard_census_sum[hazard_var + '_mean']/hazard_census_sum['area']

    hazard_census_sum = hazard_census_sum.drop([hazard_var], axis = 1)

    hazard_census_final = hazard_census_sum.merge(hazard_census_max.drop('geometry', axis = 1), on = boundary_code, how = 'left')

    hazard_census_final = boundary_id.merge(hazard_census_final.drop('geometry', axis = 1), on = boundary_code, how = 'left')

    return hazard_census_final

In [3]:
heat = gpd.read_file('https://cities-socio-economic-vulnerability.s3.eu-west-3.amazonaws.com/case-studies/campinas/campinas_heat.geojson')

In [31]:
landslide = gpd.read_file('../Data/Campinas/C4F_dados/_Perigos_climaticos/_Deslizamento/CAM_Risco_Deslizamento.shp')
landslide = landslide.set_crs(epsg = 22523)

In [32]:
flood = gpd.read_file('../Data/Campinas/C4F_dados/_Perigos_climaticos/_Alagamento/CAM_Risco_Alagamento.shp')

In [33]:
inundation = gpd.read_file('../Data/Campinas/C4F_dados/_Perigos_climaticos/_Inundaçao/CAM_Risco_Inundacao.shp')

In [2]:
census_tract = gpd.read_file('https://cities-socio-economic-vulnerability.s3.eu-west-3.amazonaws.com/case-studies/campinas/census_trac_geo.geojson')

In [37]:
heat_census_max = func_hazard_tract_agg(heat, census_tract, 'T_mean_max', 'code_tract')

In [38]:
heat_census_mean = func_hazard_tract_agg(heat, census_tract, 'T_mean_mea', 'code_tract')

In [66]:
heat_indice = func_hazard_tract_agg(heat, census_tract, 'indice', 'code_tract')

In [67]:
heat_vuln_indic = func_hazard_tract_agg(heat, census_tract, 'vuln_indic', 'code_tract')

In [68]:
heat_risco_indi = func_hazard_tract_agg(heat, census_tract, 'risco_indi', 'code_tract')

In [39]:
landslide_census = hazard_tract_agg(landslide, census_tract, 'indice_sum', 'code_tract')

In [40]:
flood_census = hazard_tract_agg(flood, census_tract, 'indice_sum', 'code_tract')

In [41]:
inundation_census = hazard_tract_agg(inundation, census_tract, 'indice_sum', 'code_tract')

In [75]:
all_hazard_tract = census_tract[['code_tract', 'geometry']]

all_hazard_tract['Heat_T_mean_max_max'] = heat_census_max['T_mean_max_max']
all_hazard_tract['Heat_T_mean_max_mean'] = heat_census_max['T_mean_max_mean']
all_hazard_tract['Heat_T_mean_mean_max'] = heat_census_mean['T_mean_mea_max']
all_hazard_tract['Heat_T_mean_mean_mean'] = heat_census_mean['T_mean_mea_mean']

all_hazard_tract['Heat_indice_max'] = heat_indice['indice_max']
all_hazard_tract['Heat_indice_mean'] = heat_indice['indice_mean']

all_hazard_tract['Heat_vuln_indic_max'] = heat_vuln_indic['vuln_indic_max']
all_hazard_tract['Heat_vuln_indic_mean'] = heat_vuln_indic['vuln_indic_mean']

all_hazard_tract['Heat_risco_indi_max'] = heat_risco_indi['risco_indi_max']
all_hazard_tract['Heat_risco_indi_mean'] = heat_risco_indi['risco_indi_mean']

all_hazard_tract['Landslide_indice_sum_max'] = landslide_census['indice_sum_max']
all_hazard_tract['Landslide_indice_sum_mean'] = landslide_census['indice_sum_mean']

all_hazard_tract['Flood_indice_sum_max'] = flood_census['indice_sum_max']
all_hazard_tract['Flood_indice_sum_mean'] = flood_census['indice_sum_mean']

all_hazard_tract['Inundation_indice_sum_max'] = inundation_census['indice_sum_max']
all_hazard_tract['Inundation_indice_sum_mean'] = inundation_census['indice_sum_mean']

In [76]:
for i in list(all_hazard_tract.columns)[2:]:
    all_hazard_tract[i] = round(all_hazard_tract[i],5)

In [78]:
all_hazard_tract.to_file('../Data/all_hazard_tract.geojson')

In [86]:
all_hazard_grid = landslide[['id', 'indice_sum', 'geometry']]

all_hazard_grid.columns = ['id', 'landslide_indice_sum', 'geometry']

heat1 = heat[['id', 'T_mean_max', 'T_mean_mea', 'indice', 'vuln_indic', 'risco_indi']]
heat1.columns = ['id', 'heat_T_mean_max', 'heat_T_mean_mea', 'heat_indice', 'heat_vuln_indic', 'heat_risco_indi']

flood1 = flood[['id', 'indice_sum']]
flood1.columns = ['id', 'flood_indice_sum']

inundation1 = inundation[['id', 'indice_sum']]
inundation1.columns = ['id', 'inundation_indice_sum']

all_hazard_grid = all_hazard_grid.merge(heat1, on = 'id', how = 'left').merge(flood1, on = 'id', how = 'left').merge(inundation1, on = 'id', how = 'left')

In [4]:
extent = census_tract[['geometry']].dissolve()

In [18]:
all_hazard_grid_tract = gpd.sjoin(all_hazard_grid, extent, 'inner')

In [88]:
all_hazard_grid_tract.to_file('../Data/all_hazard_grid.geojson')